In [ ]:
import pandas as pd
import pendulum

读数据

In [ ]:
df = pd.read_csv('data/20230222BTC_USD_1h.csv', names=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

计算缺失的序列量

In [ ]:
# UNIX Timestamp 的具体间隔
second = 60 * 1000
hour = second * 60

In [ ]:
# 计算缺失的时间并存到miss变量中
t = df['timestamp'][0]
miss = []
for i in df['timestamp']:
    while i != t:
        miss.append(t)
        t += hour
    t += hour

In [ ]:
len(miss)

In [ ]:
# 缺失百分比
len(miss) / (len(df) + len(miss)) * 100

In [ ]:
# 打印出缺失的时间段
for i, m in enumerate(miss):
    # print(m)
    if i == 0:
        print(f'start: {pendulum.from_timestamp(miss[i]/1000).to_datetime_string()}')
    elif i == len(miss) - 1:
        print(f'end: {pendulum.from_timestamp(miss[i]/1000).to_datetime_string()}')
    elif miss[i-1] == m - 60 * 1000 and miss[i+1] != m + 60 * 1000:
        print(f'end: {pendulum.from_timestamp(miss[i]/1000).to_datetime_string()}')
    elif miss[i-1] != m - 60 * 1000 and miss[i+1] == m + 60 * 1000:
        print(f'start: {pendulum.from_timestamp(miss[i]/1000).to_datetime_string()}')

In [ ]:
# 添加自然时间增加辨识度
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
# 添加当前行与上行的收盘价的变化
df['pct_change'] = df['close'].pct_change()
# 做一个偏移作为预测标签
df['label'] = df['pct_change'].shift(-1)
# 删除第一行和最后一行
df = df.iloc[1:-1]

In [ ]:
df

In [ ]:
# 查看1小时都没啥变动的值
df[df['pct_change']==0]

收益分布

In [ ]:
# 收益呈现标准差分布
df['pct_change'].plot.hist(figsize=(16, 4), bins=100)

In [ ]:
len(df.loc[df['pct_change'] > 0]) / len(df) * 100

In [ ]:
len(df.loc[df['pct_change'] == 0]) / len(df) * 100

In [ ]:
len(df.loc[df['pct_change'] < 0]) / len(df) * 100

分割训练数据和测试数据

In [ ]:
# 定义训练数据占比
ratio_training_data = 0.9
# 切割训练数据
l = int(len(df) * ratio_training_data)
training_data = df.head(l)
test_data = df.tail(len(df) - l)

In [ ]:
len(test_data)

In [ ]:
training_data.to_csv('data/out_training_data.csv')
test_data.to_csv('data/out_test_data.csv')